In [24]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Reading_API').getOrCreate()

In [27]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [5]:
import requests
import xmltodict
import json
from datetime import datetime, timedelta
import uuid

In [13]:
try:
    job_id = str(uuid.uuid1())
    job_start_time = str(datetime.now())
    
    user = ''
    password = ''
    url = 'https://www.washingtonpost.com/arcio/news-sitemap/'
    table_name = ''
    batch_ctrl_key = ''
    
    audit_json = {
        "job_id": job_id,
        "attributes": [
            {
                "name": "dim_table",
                "value": table_name
            }
        ],
        "job_start_time": job_start_time,
    }
    stats = {}
except Exception as e:
#     (
#         job_audit_handler(
#             audit_json = audit_json,
#             cmd = 4,
#             batch_ctrl_key = batch_ctrl_key,
#             status = 'FAILURE',
#             exception = e,
#             stats = stats
#         )
#     )
    raise e

In [14]:
try:
    response = requests.get(url)
    if response.status_code != 200:
#         job_audit_handler(
#             audit_json = audit_json,
#             cmd = 4,
#             batch_ctrl_key = batch_ctrl_key,
#             status = 'FAILURE',
#             exception = str(response.status_code) + ' API request to get the details failed.',
#             stats = stats
#         )
#         dbutils.notebook.exit("Exiting Job, as API call failed")
        print("API call failed.")
except Exception as e:
    (
        job_audit_handler(
            audit_json = audit_json,
            cmd = 4,
            batch_ctrl_key = batch_ctrl_key,
            status = 'FAILURE',
            exception = e,
            stats = stats
        )
    )
    raise e

In [15]:
print(response.text)

<?xml version="1.0" encoding="UTF-8"?>

<urlset xmlns="http://www.sitemaps.org/schemas/sitemap/0.9"
    xmlns:news="http://www.google.com/schemas/sitemap-news/0.9"
    xmlns:video="http://www.google.com/schemas/sitemap-video/1.1">
<url>
         <loc>https://www.washingtonpost.com/politics/ohio-primaries-may-signal-mood-of-2022-midterm-race/2021/08/03/1c9418f4-f44c-11eb-a636-18cac59a98dc_story.html</loc>
         <lastmod>2021-08-04T03:29:34.000Z</lastmod>
         <news:news>
            <news:publication>
                <news:name>Washington Post</news:name>
                <news:language>en</news:language>
            </news:publication>
            
            <news:publication_date>2021-08-04T03:29:34.000Z</news:publication_date>
            <news:title><![CDATA[Trump-backed Carey, centrist Brown win Ohio US House races]]></news:title>
            <news:keywords><![CDATA[US--Election 2021-Ohio-House, General news, Government and politics, State legislature, Legislature, Politica

In [21]:
def look_up(setval, val):
    try:
        if isinstance(setval, list) == True:
            a = []
            for i in setval:
                a.append(i[val])
            return a
        else:
            return setval[val]
    except Exception as e:
        return 'NA'

In [52]:
try:
    my_list = []
    response_text = response.text
    json_data = xmltodict.parse(response_text)
    json_data = json.loads(json.dumps(json_data))
    for item in json_data['urlset']['url']:
        my_list.append((look_up(item, 'loc'),\
                       look_up(item, 'lastmod'),\
#                        look_up(item, 'news:news'),\
                       look_up(item['news:news']['news:publication'], 'news:name'),\
                       look_up(item['news:news']['news:publication'], 'news:language'),\
                       look_up(item['news:news'], 'news:publication_date'),\
                       look_up(item['news:news'], 'news:title'),\
                       look_up(item['news:news'], 'news:keywords'),\
                       look_up(item, 'changefreq')))

    my_Columns = ["Location", "Last_Modified", "Name", "Language", "Publsh_Dt", "Title", "Keywords", "ChangeFreq"]
    my_DF = spark.createDataFrame(data=my_list, schema = my_Columns)
    my_DF.createOrReplaceTempView('my_vw')
#     my_DF.show()
    my_DF.printSchema()
    
except Exception as e:
    raise e

root
 |-- Location: string (nullable = true)
 |-- Last_Modified: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Language: string (nullable = true)
 |-- Publsh_Dt: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Keywords: string (nullable = true)
 |-- ChangeFreq: string (nullable = true)



In [55]:
my_DF.columns

['Location',
 'Last_Modified',
 'Name',
 'Language',
 'Publsh_Dt',
 'Title',
 'Keywords',
 'ChangeFreq']

In [61]:
my_DF.select(my_DF.Location).filter(my_DF.Location.contains('sports')).show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Location                                                                                                                                                            |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|https://www.washingtonpost.com/sports/olympics/2021/08/03/tokyo-olympics-2021-live-updates-highlights-day-12/                                                       |
|https://www.washingtonpost.com/sports/mlb/mariners-beat-rays-for-the-6th-straight-game-this-season/2021/08/03/277839ec-f4d0-11eb-a636-18cac59a98dc_story.html       |
|https://www.washingtonpost.com/sports/mlb/cease-blanks-royals-for-6-innings-white-sox-win-7-1/2021/08/03/4cfc3b02-f4d3-11eb-a636-18cac59a98dc_story.html            